In [ ]:
import pandas as pd # pour les séries temporelles
import matplotlib.pyplot as plt # pour les plot
import numpy as np
from copy import deepcopy
from BuysBallot import BuysBallotModel  # module python pour la méthode de Buys Ballot
from datetime import datetime  # gestion des dates
import scipy.stats  # outils statistiques
import statsmodels.api as sm # outils pour les séries temporelles

In [ ]:
# Suppression des relevés pour les 29/02 des années bissextiles
path = "Data/ukcp09_gridded-land-obs-daily_timeseries_mean-temperature_000000E_500000N_19600101-20161231.csv"
data = pd.read_csv(path, header=[0,1], index_col=0, parse_dates=True)
dates_to_remove = [datetime.date(datetime.strptime(str(y)+"-02-29", '%Y-%m-%d')) for y in range(1960, 2017, 4)]
data = data.drop(dates_to_remove)
# Lecture de la série des résidus (enregistrée indépendemment)
serie = pd.read_csv("BB22500-547500.csv", header=None)[1]
serie.index = data.index
serie.head()

In [ ]:
# On ne garde qu'une valeur sur 5
serie = serie.values[list(range(0,len(serie),5))]

In [ ]:
# On ne garde qu'1/5 des données pour la calibration du modèle
donnees = serie[:int(len(serie)/5)]
len(donnees)

## Approche naïve

In [ ]:
# On cherche à modéliser la distribution par une loi connue. Vue la forme, on peut s'intéresser à une gaussienne
mean_gaussian, std_gaussian = scipy.stats.norm.fit(donnees)

In [ ]:
x = np.linspace(-8,8,500)
plt.figure(figsize=(10,10))
plt.hist(donnees, bins = int(2*832**(1/3)), normed=True, label = "Distribution empirique")
plt.plot(x, scipy.stats.norm.pdf(x, loc = mean_gaussian, scale = std_gaussian), label = "Estimation par une loi normale")
plt.legend()
plt.show()

In [ ]:
# Calcul du quantile d'ordre 10**(-3) pour le modèle approchant les données
q = scipy.stats.norm.ppf(1-10**(-3), loc = mean_gaussian, scale = std_gaussian)
q

In [ ]:
np.where(serie>q)

In [ ]:
np.where(-serie>q)

Une seule valeur sur plus de 4000 données dépasse le quantile d'ordre 10^-5. Tout porte à croire que la queue de distribution est plus fine qu'une gaussienne. Regardons le qqplot:

In [ ]:
res = scipy.stats.probplot(serie, plot=plt)

Le qqplot confirme ces hypothèses pour ce qui est de la distribution pour les très grandes valeurs (queue de distribution plus légèrement plus fine). En revanche, la queue de distribution sur les valeurs négatives semble beaucoup plus épaisse que la gaussienne.

## Statistiques des valeurs extrêmes

### Estimateur de Hill

In [ ]:
# choix de k pour les valeurs sup
def alpha_Hill(donnees, k):
    n = len(donnees)
    sorted_data = sorted(donnees)
    xi = 1/k*np.sum(np.log(sorted_data[n-k:]/sorted_data[n-k]))
    return 1/xi

In [ ]:
# Pour les valeurs sup
alpha = np.array([alpha_Hill(donnees, k) for k in range(20,200)])
alpha_sup = alpha + 1/np.sqrt(np.arange(20,200))*alpha**2
alpha_inf = alpha - 1/np.sqrt(np.arange(20,200))*alpha**2
plt.figure(figsize=(10,10))
plt.plot(range(20,200), alpha, label = "alpha")
plt.plot(range(20,200), alpha_sup, color='r', label="IC à 95%")
plt.plot(range(20,200), alpha_inf, color='r')
plt.legend()
plt.show()

In [ ]:
# Pour les valeurs sup
alpha2 = np.array([alpha_Hill(-donnees, k) for k in range(20,200)])
alpha_sup2 = alpha2 + 1/np.sqrt(np.arange(20,200))*alpha2**2
alpha_inf2 = alpha2 - 1/np.sqrt(np.arange(20,200))*alpha2**2
plt.figure(figsize=(10,10))
plt.plot(range(20,200), alpha2, label = "alpha")
plt.plot(range(20,200), alpha_sup2, color='r', label="IC à 95%")
plt.plot(range(20,200), alpha_inf2, color='r')
plt.legend()
plt.show()

In [ ]:
alpha[30]

In [ ]:
alpha2[30]

In [ ]:
# Estimation des quantiles
def quantiles(donnees, k):
    alphainf = alpha_Hill(-donnees, k)
    alphasup = alpha_Hill(donnees, k)
    n = len(donnees)
    sorted_donnees = sorted(donnees)
    q_sup = (1+(n/k*(1-10**(-3)))**(-1/alphasup))*sorted_donnees[n-k]
    q_inf = (1+(n/k*(1-10**(-3)))**(-1/alphainf))*sorted_donnees[k]
    return q_sup, q_inf

In [ ]:
quantiles_sup = []
quantiles_inf = []
for k in range(1,50):
    qs, qi = quantiles(donnees, k)
    quantiles_sup.append(qs)
    quantiles_inf.append(qi)

In [ ]:
plt.plot(range(1,50), quantiles_sup)
plt.show()

In [ ]:
plt.plot(range(1,50), quantiles_inf)
plt.show()

In [ ]:
# En utilisant toutes les données
quantiles_sup = []
quantiles_inf = []
for k in range(1,50):
    qs, qi = quantiles(serie, k)
    quantiles_sup.append(qs)
    quantiles_inf.append(qi)

In [ ]:
plt.plot(range(1,50), quantiles_sup)
plt.show()

In [ ]:
plt.plot(range(1,50), quantiles_inf)
plt.show()

### Estimateur de Pickands

In [ ]:
def pickands(donnees, k):
    n = len(donnees)
    sorted_data = sorted(donnees)
    return 1/np.log(2)*np.log((sorted_data[n-k]-sorted_data[n-2*k])/(sorted_data[n-2*k]-sorted_data[n-4*k]))

In [ ]:
xi = [pickands(serie, k) for k in range(10,100)]
plt.plot(range(10,100), xi)
plt.show()

### Méthode POT (Picks Over Threshold)

In [ ]:
def e(donnees, u):
    return np.mean(donnees[np.where(donnees>u)[0]] - u)

In [ ]:
# Espérance des excès e(u) à droite
x = np.linspace(1,6, 100)
y = []
for u in x:
    y.append(e(serie, u))
plt.plot(x,y)
plt.title("Espérance des excès en fonction du seuil u pour les positifs")
plt.show()

In [ ]:
# Espérance des excès e(u) à gauche
x = np.linspace(1,6, 100)
y = []
for u in x:
    y.append(e(-serie, u))
plt.plot(x,y)
plt.title("Espérance des excès en fonction du seuil u pour les négatifs")
plt.show()

In [ ]:
# Choix du seuil u = 4 (début de la "tendance linéaire") à droite et u = -5 pour les négatifs
# On fit par maximum de vriasemblance une distribution de Pareto généralisée sur les données dépassant le seuil
seuil1 = 5
seuil2 = 4

In [ ]:
# Calcul des quantiles à gauche (inf)
xi, loc, beta = scipy.stats.genpareto.fit(-serie[np.where(-serie>seuil1)[0]]-seuil1, loc = 0)
n = len(donnees)
sorted_data = sorted(-serie)
q_inf = - seuil1 + beta/xi*((n/len(np.where(-serie>seuil1)[0])*(1-10**(-3)))**(-beta)-1)
print("xi = "+str(xi))
print("(alpha = "+str(1/xi)+")")
print("Quantile à l'ordre 10**-3 : "+str(q_inf))

In [ ]:
# Calcul des quantiles à droite (sup)
xi, loc, beta = scipy.stats.genpareto.fit(serie[np.where(serie>seuil2)[0]]-seuil2, loc = 0)
sorted_data = sorted(serie)
q_sup = seuil2 + beta/xi*((n/len(np.where(serie>seuil2)[0])*(1-10**(-3)))**(-beta)-1)
print("xi = "+str(xi))
print("(alpha = "+str(1/xi)+")")
print("Quantile à l'ordre 10**-3 : "+str(q_sup))